In [119]:
import torch 
from torchtext import data, datasets
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import spacy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
#In this version we will implement the bag of tricks for efficient text calssification


In [7]:
#https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb

In [8]:
#We first need to define a function calle Bigrams
#Bigrams are two work snippets of a given sentence
#they are words that appear consecutively within a sentence
#In bag of Tricks you append the n-gram of an input sentence and append it to the end of the sentence


In [13]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [14]:
#example of bi grams
generate_bigrams(['This', 'film', 'is', 'great'])


['This', 'film', 'is', 'great', 'film is', 'is great', 'This film']

In [16]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams,tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [19]:
train_data, test_data = datasets.IMDB.splits(TEXT,LABEL)

train_data, val_data = train_data.split(random_state = random.seed(SEED))

In [21]:

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)


In [35]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
                                            (train_data, val_data, test_data),
                                            batch_size = BATCH_SIZE,
                                                device = device)

In [20]:
%%HTML
<h3> Build The Model </h3>

In [23]:
#This model will have two layers
#firt layer calculates word embeddings, then second layer calcualtes the average
#of all the wor embeddings
#final FC layer is a linear layer
#No RNN
#We will be using avg_pool2d function to on the word vs wordembedding matric
#imagine your sentence is lined up vertically and the wrod embeddings horizontal
#the avg_pool2d takes the average word embedding for each of the n-embeddings 
#this tensor is then passed to linera layer

In [24]:
class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        #text = [sentence length, batch size]
        embedded = self.embedding(text)
        
        #embedded = [sentence len, batchs ize, emb dim]
        
        embedded = embedded.permute(1,0,2)
        
        #embedded = [batch size, sent len, embdim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)
        
        #pooled = batchsize, embed_dim 
        
        return self.fc(pooled)
        

In [25]:
#Create an instance of the fAST text class

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM =1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM,EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [26]:
#Count parameters of model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad_)

In [27]:
print(f'The model has {count_parameters(model): ,} trainable parameters')

The model has  2,500,301 trainable parameters


In [28]:
#Much less parameters than previous models


In [29]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4835,  1.2209, -0.8538,  ..., -0.6372, -0.4933, -0.5874],
        [ 1.8895, -2.5150,  0.1815,  ...,  0.4264, -1.0151,  0.1059],
        [ 0.9233,  0.3456,  0.1981,  ..., -0.7031, -0.3052,  0.7518]])

In [31]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [33]:
optimizer = optim.Adam(model.parameters())


In [36]:
loss_fn = nn.BCEWithLogitsLoss()

model = model.to(device)
loss_fn = loss_fn.to(device)

In [54]:
#Calculate accuracy

def binary_accuracy(preds, y):
    #round pred to closes int
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = rounded_preds == y.float()
    
    acc = correct.sum() /len(correct)
    
    return acc

In [87]:
#Define Training modle
def train(model, iterator, optimizer, loss_fn):
    
    epoch_loss = 0
    epoch_acc  = 0
    
    model.train()
    correct =0
    
    for batch in iterator:
        
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = loss_fn(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss+= loss.item()
        epoch_acc+= acc.item()
        
        correct += acc #(predictions ==batch.label).float().sum()
        
        #print('Correct : ', correct)
        
    return epoch_loss /len(iterator), epoch_acc/len(iterator), correct 
        

In [88]:
def eval(model, iterator, loss_fn):
    
    epoch_loss = 0 
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss+=loss.item()
            
            epoch_acc +=acc.item()
            
            
        return epoch_loss/len(iterator), epoch_acc/ len(iterator)

In [89]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [99]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    

    
    start_time = time.time()
    
    train_loss, train_acc, correct = train(model, train_iterator, optimizer, loss_fn)
    
    val_loss, val_acc = eval(model, val_iterator, loss_fn)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    #print('Correct: ', correct)

Epoch: 01 | Epoch Time: 1m 27s
	Train Loss: 0.185 | Train Acc: 2.55%
	 Val. Loss: 0.514 |  Val. Acc: 0.00%
Correct:  tensor(7)
Epoch: 02 | Epoch Time: 1m 47s
	Train Loss: 0.173 | Train Acc: 3.65%
	 Val. Loss: 0.530 |  Val. Acc: 0.00%
Correct:  tensor(10)
Epoch: 03 | Epoch Time: 1m 47s
	Train Loss: 0.162 | Train Acc: 5.47%
	 Val. Loss: 0.554 |  Val. Acc: 0.00%
Correct:  tensor(15)
Epoch: 04 | Epoch Time: 1m 46s
	Train Loss: 0.152 | Train Acc: 5.84%
	 Val. Loss: 0.567 |  Val. Acc: 0.00%
Correct:  tensor(16)
Epoch: 05 | Epoch Time: 1m 48s
	Train Loss: 0.143 | Train Acc: 9.85%
	 Val. Loss: 0.588 |  Val. Acc: 0.00%
Correct:  tensor(27)
Epoch: 06 | Epoch Time: 2m 4s
	Train Loss: 0.134 | Train Acc: 10.95%
	 Val. Loss: 0.600 |  Val. Acc: 0.00%
Correct:  tensor(30)
Epoch: 07 | Epoch Time: 2m 1s
	Train Loss: 0.126 | Train Acc: 12.41%
	 Val. Loss: 0.630 |  Val. Acc: 0.00%
Correct:  tensor(34)
Epoch: 08 | Epoch Time: 1m 56s
	Train Loss: 0.118 | Train Acc: 14.23%
	 Val. Loss: 0.640 |  Val. Acc: 0.0

In [100]:
%%HTML

<h3> User Input </h3>



In [101]:
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()


In [102]:
predict_sentiment(model, 'This movie is awful')

0.0

In [103]:
predict_sentiment(model, 'This movie is amazing')

1.0

In [107]:
predict_sentiment(model, 'This movie is average')

0.9999991655349731

In [109]:
%%HTML 
<h3> Connecting Tiwtter API </h3>

In [113]:
#enter twitter api keys

consumer = '9CdP5ODjeCyT4kNWbkyvo13XS'
consumer_secret = 'vslbOEeTTLUjKedmT7Mkb9MBGBrrJ8bHNDuYn5AX2I58WjfdsD'

access_token = '22669732-hniecW89vTM8Hyi2voU5MNqXBMEF6qdTGwBeC0Zny'
access_secret= 'Ltsj90B7moR0qZ2oNTBlbe3F3oyJRwL3qY6vt3XEC4EzU'


In [128]:
auth = tweepy.OAuthHandler(consumer, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
#Create tweet scraping function

def getTweets(search_params, date_since,numTweets, runs):
    #define datafrome to store date
    
    df = pd.DataFrame(columns= ['username', 'acctdesc', 'location', 'following', 'followers','totalTweets', 'usercreatedts', 'tweetcreatedts', 'retweetcount', 'text','hashtags'])
    start_time = time.time()
    
    #collect tweets using Cursor object
    
    for i in range(runs):
        #start_time = time.time()
        
        tweets = tweepy.Cursor(api.search, q = search_params, lang = 'en', since = date_since, tweet_mode = 'extended').items(numTweets)
        
        
        #store in list
        tweet_list = [tweet for tweet in tweets]
        
    #iter through tweets and pull specific info
    noTweets = 0
    for tweet in tweet_list:
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totalTweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts= tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        try: 
            text = tweet.retweeted_status.full_text
        except AttributeError:
            #not a retweet
            text = tweet.full_text
        
        ith_tweet = [username, acctdesc, location, following, followers, totalTweets, usercreatedts,tweetcreatedts, retweetcount, text, hashtags]
        
        df.loc[len(df)] = ith_tweet
        
        noTweets +=1
        
        end_time = time.time()
        duration = round((end_time -start_time)/60,2)
        
    print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
    print('time take for {} run to complete is {} mins'.format(i+1, duration))
        
        #time.sleep(920) #15mins
              
        #obntaine timestamp
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
              
        #save dataframe 
        
    df.to_csv('.\Tweets', 'Tweets' + to_csv_timestamp + '.csv', index=False)
        
    print("Scraping complete")
        
        

In [139]:
#init search param 

search_params = '$EWT OR #EnergyWeb OR #EWT'
date_since = '2020-01-01'
numTweets = 2500
runs = 1

#call funtion 

getTweets(search_params, date_since, numTweets,runs)

no. of tweets scraped for run 1 is 1
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 2
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 3
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 4
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 5
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 6
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 7
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 8
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 9
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 10
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 11
time take for 1 run to complete is 0.54 mins
no. of tweets scraped for run 1 is 12
time take for 1 run to complete is 0.54 mins
no. of tweets

no. of tweets scraped for run 1 is 104
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 105
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 106
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 107
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 108
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 109
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 110
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 111
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 112
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 113
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 114
time take for 1 run to complete is 0.55 mins
no. of tweets scraped for run 1 is 115
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 205
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 206
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 207
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 208
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 209
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 210
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 211
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 212
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 213
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 214
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 215
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 216
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 350
time take for 1 run to complete is 0.56 mins
no. of tweets scraped for run 1 is 351
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 352
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 353
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 354
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 355
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 356
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 357
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 358
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 359
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 360
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 361
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 451
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 452
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 453
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 454
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 455
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 456
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 457
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 458
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 459
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 460
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 461
time take for 1 run to complete is 0.57 mins
no. of tweets scraped for run 1 is 462
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 574
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 575
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 576
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 577
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 578
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 579
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 580
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 581
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 582
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 583
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 584
time take for 1 run to complete is 0.58 mins
no. of tweets scraped for run 1 is 585
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 673
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 674
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 675
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 676
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 677
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 678
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 679
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 680
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 681
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 682
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 683
time take for 1 run to complete is 0.59 mins
no. of tweets scraped for run 1 is 684
time take for 1 run to complete is 0.

no. of tweets scraped for run 1 is 802
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 803
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 804
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 805
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 806
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 807
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 808
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 809
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 810
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 811
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 812
time take for 1 run to complete is 0.6 mins
no. of tweets scraped for run 1 is 813
time take for 1 run to complete is 0.6 mins
no. 

In [141]:
ewt = pd.read_csv(r'.\Tweets\Tweets_EWT.csv')

In [143]:
ewt.head()

,username,acctdesc,location,following,followers,totalTweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags
0,EightyNineWaves,"Professional forecasts, analytics and trading ...",NaN,285,2057,1285,2016-07-10 12:56:53,2020-07-11 18:55:08,0,$MOEX index\nTimeframe: 1H\n\nWe were too bear...,"[{'text': 'moex', 'indices': [174, 179]}, {'te..."
1,dimdimdimma,$EWT,NaN,176,34,124,2013-12-15 16:57:21,2020-07-11 18:54:28,5,@binance $EWT \n\nLet’s be transparent here: i...,[]
2,fswlgkkt1,NaN,NaN,7,2,5,2020-06-23 18:30:35,2020-07-11 18:42:39,11,Closing in on shitcoins and making the grand e...,"[{'text': 'soon', 'indices': [84, 89]}]"
3,Bullrun_Gravano,Investor | Margin Trader | $EWT | $DMG Underbo...,NaN,105,3367,8767,2018-05-23 19:14:21,2020-07-11 18:35:00,0,@Dentoshi93 @RookieXBT Cheers ...\n\n$DMG @DMM...,[]
4,Smartcontracts3,ENG/FIN\nWe are also in Facebook: https://t.co...,NaN,18,1,16,2020-06-29 10:26:50,2020-07-11 18:31:16,5,@binance $EWT \n\nLet’s be transparent here: i...,[]


In [144]:
new_data = ewt['text']


0      $MOEX index\nTimeframe: 1H\n\nWe were too bear...
1      @binance $EWT \n\nLet’s be transparent here: i...
2      Closing in on shitcoins and making the grand e...
3      @Dentoshi93 @RookieXBT Cheers ...\n\n$DMG @DMM...
4      @binance $EWT \n\nLet’s be transparent here: i...
                             ...                        
928    @bitbns What's the process to get a coin liste...
929    Maby this confirms all the speculation and hyp...
930    @bitbns What's the process to get a coin liste...
931    Like my tweet if you are a @UnibrightIO $ubt f...
932    How to store $EWT on @metamask_io\n\nhttps://t...
Name: text, Length: 933, dtype: object

In [163]:
for i in new_data:
    
    print(predict_sentiment(model, i))

0.9998632669448853
0.1975666731595993
5.583407540473457e-22
1.3509435348169063e-07
0.1975666731595993
0.1910361647605896
7.868144054555931e-19
0.1910361647605896
0.1910361647605896
1.0507682235167543e-13
4.294026246916285e-17
5.583407540473457e-22
0.5767343640327454
5.860636411933462e-13
4.421638046992801e-17
5.860636411933462e-13
5.860636411933462e-13
5.860636411933462e-13
3.1368301733891713e-06
5.860636411933462e-13
5.583407540473457e-22
5.860636411933462e-13
5.860636411933462e-13
5.860636411933462e-13
5.860636411933462e-13
5.860636411933462e-13
4.294026246916285e-17
3.1368301733891713e-06
0.011719511821866035
1.4495522344005277e-21
4.294026246916285e-17
5.583407540473457e-22
0.1975666731595993
0.014382944442331791
1.4495522344005277e-21
0.00025660195387899876
4.294026246916285e-17
1.4495522344005277e-21
4.294026246916285e-17
7.932398148113862e-05
1.1856865133850222e-16
1.1856865133850222e-16
1.1856865133850222e-16
1.4495522344005277e-21
1.1856865133850222e-16
1.1856865133850222e-16


1.2070178840792312e-14
1.2070178840792312e-14
0.9999998807907104
2.0740835680044256e-06
0.999956488609314
0.999956488609314
2.7981160818273468e-14
0.999956488609314
1.5621978155522243e-09
8.443188509454913e-11
1.7327361249243117e-15
1.0
1.0
5.770781629881653e-30
1.0
1.5621978155522243e-09
1.0
4.118299035127393e-10
1.5621978155522243e-09
1.0
1.5621978155522243e-09
1.5621978155522243e-09
5.429837475133504e-16
5.429837475133504e-16
7.735238050404892e-18
1.5621978155522243e-09
8.269205764577237e-09
8.269205764577237e-09
1.0
1.5621978155522243e-09
0.0
1.5621978155522243e-09
1.5621978155522243e-09
5.675324439829653e-19
1.0
3.4692552650095188e-12
0.0
3.4692552650095188e-12
1.7327361249243117e-15
3.4692552650095188e-12
1.0
1.5621978155522243e-09
1.5621978155522243e-09
9.117595023902167e-28
9.117595023902167e-28
1.5621978155522243e-09
1.7393555026501417e-05
1.5621978155522243e-09
1.5621978155522243e-09
1.5621978155522243e-09
1.5621978155522243e-09
1.5621978155522243e-09
1.5621978155522243e-09
1

4.862831970016153e-19